In [3]:
import scipy
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [4]:
df = pd.read_csv("Data.csv", sep=";")
df.dtypes

Direksi Perempuan    int64
Direksi Laki-Laki    int64
Harga penutupan      int64
Total Shares         int64
Total Utang          int64
Total Asset          int64
dtype: object

In [5]:
X = (df['Direksi Perempuan'])/(df['Direksi Perempuan'] + df['Direksi Laki-Laki'])
#X = df["Direksi Perempuan"]
nilai_pasar = df["Total Shares"] * df["Harga penutupan"]
np_plus_liability = nilai_pasar + df["Total Utang"]
Y = np_plus_liability / df["Total Asset"]
# Y = nilai_pasar / df["Total Asset"]

data = pd.DataFrame({"X":X, "Y":Y})
X_mean = np.mean(data["X"])
X_std = np.std(data["X"])
Y_mean = np.mean(data["Y"])
Y_std = np.std(data["Y"])

In [6]:
data_sample = pd.DataFrame({"X": X, "Y": Y})
data_sample.describe()

,X,Y
count,150.000000,150.000000
mean,0.184803,1.297296
std,0.150672,2.019696
min,0.000000,0.159993
25%,0.000000,0.930708
50%,0.190909,0.978282
75%,0.282468,1.141656
max,0.750000,18.327231


In [7]:
# data_sample.drop(data_sample[data_sample["X"] == 0].index, inplace=True)
# np.size(np.where(data_sample["X"] == 0))

In [8]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.06500
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.799
Time:                        20:02:41   Log-Likelihood:                -317.75
No. Observations:                 150   AIC:                             639.5
Df Residuals:                     148   BIC:                             645.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3492      0.262      5.143      0.000       0.831       1.868
X             -0.2809      1.102     -0.255      0.799      -2.458       1.896
==============================================================================
Omnibus:                      257.367   Durbin-Watson:                   0.889
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27452.762
Skew:                           7.990   Prob(JB):                         0.00
Kurtosis:                      67.320   Cond. No.                         6.89
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [9]:
bX = data_sample["X"] * result.params[0]
abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [10]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.018
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.315
Time:                        20:02:41   Log-Likelihood:                -305.38
No. Observations:                 150   AIC:                             614.8
Df Residuals:                     148   BIC:                             620.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6728      0.242      2.785      0.006       0.195       1.150
X              1.0236      1.014      1.009      0.315      -0.981       3.028
==============================================================================
Omnibus:                      262.529   Durbin-Watson:                   0.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            30220.631
Skew:                           8.268   Prob(JB):                         0.00
Kurtosis:                      70.541   Cond. No.                         6.89
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
XQ3,XQ1 = np.percentile(data_sample["X"], [75 ,25])
XIQR = XQ3 - XQ1 

In [12]:
lower_X = XQ1 - (1.5 * XIQR)
upper_X = XQ3 + (1.5 * XIQR)

In [13]:
data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)]

56    0.75
Name: X, dtype: float64

In [14]:
data_sample.drop(data_sample["X"][(data_sample["X"] < lower_X) | (data_sample["X"] > upper_X)].index, inplace=True)

In [15]:
YQ3,YQ1 = np.percentile(data_sample['Y'], [75 ,25])
YIQR = YQ3 - YQ1 

In [16]:
lower_Y = YQ1 - (1.5 * YIQR)
upper_Y = YQ3 + (1.5 * YIQR)

In [17]:
data_sample.drop(data_sample["Y"][(data_sample["Y"] < lower_Y) | (data_sample["Y"] > upper_Y)].index, inplace=True)

In [18]:
data_sample.describe()

,X,Y
count,126.000000,126.000000
mean,0.197119,1.009978
std,0.141369,0.125106
min,0.000000,0.688744
25%,0.100000,0.933108
50%,0.200000,0.971672
75%,0.285714,1.061116
max,0.545455,1.385464


In [19]:
X_with_constant = sm.add_constant(data_sample["X"])
result = sm.OLS(data_sample["Y"], X_with_constant).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.668
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.105
Time:                        20:02:41   Log-Likelihood:                 84.959
No. Observations:                 126   AIC:                            -165.9
Df Residuals:                     124   BIC:                            -160.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9847      0.019     51.694      0.000       0.947       1.022
X              0.1284      0.079      1.633      0.105      -0.027       0.284
==============================================================================
Omnibus:                       18.090   Durbin-Watson:                   0.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.108
Skew:                           0.974   Prob(JB):                     2.61e-05
Kurtosis:                       3.473   Cond. No.                         7.38
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
bX = data_sample["X"] * result.params["X"]

In [21]:
# Y = a + bX + e
# e = Y - a - bX

abs_residual = np.abs(data_sample["Y"] - result.params["const"] - bX)

In [22]:
hetero_result = sm.OLS(abs_residual, X_with_constant).fit()
hetero_result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7508
Date:                Thu, 11 May 2023   Prob (F-statistic):              0.388
Time:                        20:02:42   Log-Likelihood:                 148.26
No. Observations:                 126   AIC:                            -292.5
Df Residuals:                     124   BIC:                            -286.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0899      0.012      7.796      0.000       0.067       0.113
X              0.0412      0.048      0.867      0.388      -0.053       0.135
==============================================================================
Omnibus:                       40.046   Durbin-Watson:                   1.219
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.410
Skew:                           1.449   Prob(JB):                     3.12e-16
Kurtosis:                       5.282   Cond. No.                         7.38
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""